<a href="https://colab.research.google.com/github/filsto/GAN/blob/main/GAN_book_aurelien_geron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential


import matplotlib.pyplot as plt
import numpy as np

In [11]:
codings_size = 30

generator = keras.models.Sequential ([
    keras.layers.Dense(100,activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation='selu'),
    keras.layers.Dense(28*28, activation="sigmoid"),
    keras.layers.Reshape([28,28])                                   
])

discriminator = keras.models.Sequential ([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")                                          

])

gan=keras.models.Sequential([generator, discriminator])

In [12]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable=False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [13]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

NameError: ignored

In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
  generator, discriminator = gan.layers
  for epoch in range (n_epochs):
    for X_batch in dataset:
      #Phase 1 - entraînement du discriminateur
      noise = tf.random.normal(shape=[batch_size, codings_size])
      generated_images = generator(noise)
      X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
      y1=tf.constant([0.] * batch_size + [[1.]] * batch_size)
      discriminator.trainable = True
      discriminator.train_on_batch(X_fake_and_real, y1)
      #Phase 2 - entraînement du générateur
      noise = tf.random.normal(shape=[batch_size, codings_size])
      y2 = tf.constant([1.] * batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y2)